In [1]:
import streamlit as st
import langchain, os, openai, dotenv

import re

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage

import datetime

from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma

import chromadb
import dotenv, time

import os, glob
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

print(openai.__version__)



1.3.7


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
embeddings = OpenAIEmbeddings()

In [3]:
base_path = "./vectorDB/"

In [7]:
vectordb = Chroma(persist_directory=base_path,embedding_function=embeddings)
print(len(vectordb.get()["ids"]),"elements already stored.")

6643 elements already stored.


In [8]:
def create_agent_chain(llm):
    chain = load_qa_chain(llm, chain_type="stuff")
    return chain

In [10]:
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models import ChatOpenAI

In [17]:
import hashlib

In [46]:
def get_llm_response(query,vectordb,temperature=0.1,k=10):
    F = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    Q = query+","+str(temperature)+","+str(k)
    Q = hashlib.md5(Q.encode()).hexdigest()
    print(Q)
    FILE = "cache/"+Q+".md"
    if os.path.isfile(FILE):
        with open(FILE,"r") as f:
            A =f.read()
            answer = A.split("\n\n---\n\n>A:\n")[-1].strip()
            docs = A.split("\n\n---\n\n>A:\n")[-2].split("\n\n---\n\nD:\n")[-1].strip()
            docs = re.findall("\'article\': \'(.*?)\'}", docs, re.DOTALL)
    else:
        llm = ChatOpenAI(
            # models : https://platform.openai.com/docs/models
            temperature=temperature,
            model="gpt-3.5-turbo-1106"
        )
        chain = create_agent_chain(llm)
        matching_docs = vectordb.similarity_search(query,k)
        answer = chain.run(input_documents=matching_docs, question=query)
        docs = [x.metadata["article"] for x in matching_docs]
        with open(FILE,"w") as f:
            f.write(">Q:\n"+query +"\n\n---\n\nD:\n"+str(matching_docs)+ "\n\n---\n\n>A:\n"+answer)
    return answer, docs

In [47]:
a, d = get_llm_response("What is UV-C",vectordb,temperature=0.1,k=10)
a

e9742f8acc4e2fe49f1a3ee33998721c


'UV-C, or ultraviolet germicidal irradiation, is a disinfection method that uses short-wavelength ultraviolet rays to kill or inactivate microorganisms by destroying their nucleic acids and disrupting their DNA, rendering them unable to perform their cellular functions. UV-C is used in a variety of applications, such as purifying food, air, and water. It is particularly effective against airborne bacteria and viruses, including the coronavirus. UV-C radiation is typically produced by fluorescent lamps or gas discharge lamps and is used in various settings, including upper-room fixtures, air handling units, and air purification systems. UV-C radiation is known to be harmful to human skin and eyes and can accelerate the degradation of certain materials, so direct exposure to humans and sensitive materials should be avoided.'